In [12]:
import collections
import numpy as np
import pandas as pd
import re
import glob
import json

from argparse import Namespace

from torchtext.utils import download_from_url, extract_archive

In [13]:
dataset_tar = download_from_url('https://drive.google.com/uc?id=1k7GfVRqrHFK00ABkit0oGQo62fCakMSZ', root='.data/')
extracted_files = extract_archive(dataset_tar)

In [14]:
args = Namespace(
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv=".data/faq_with_splits.csv",
    seed=1234,
    min_questions_per_category=3000
)

In [15]:
def find_files(path):
    return glob.glob(path)

def read_json(filename):
    with open(filename) as json_file:
        data = json.load(json_file)
    return data

In [16]:
json_data = []

for file in find_files('data/gyakori_*'):
    print("Reading file:", file)
    json_data.append(read_json(file))
    
faq = pd.DataFrame()

for data in json_data:
    frames  = [faq, pd.DataFrame(data)]
    faq = pd.concat(frames).reset_index(drop=True)

Reading file: data/gyakori_egeszseg_20000
Reading file: data/gyakori_egeszseg
Reading file: data/gyakori_szamitastechnika
Reading file: data/gyakori_szorakozas_30000
Reading file: data/gyakori_allatok_14000


In [17]:
faq.head()

,keywords,hosszu_kerdes,valasz,rovid_kerdes,kategoriak
0,"[inzulinrezisztencia, diéta, reggeli]",Inzulinrezisztencia diétában mennyire kell kom...,Eléggé. Az ébredéstől egy órán belül meg kelle...,Inzulinrezisztencia diétában mennyire kell kom...,"[Egészség, Táplálkozás]"
1,"[nőgyógyászat, kivizsgálás, fájdalom, magánren...","Ha Borsod megyei vagyok, attól még mehetek mag...","Akárkihez mehetsz, akárhol.","Ha Borsod megyei vagyok, attól még mehetek mag...","[Egészség, Nők egészsége]"
2,"[növény, halál]","Melyik az a növény amelyet, ha bent tartasz a ...",Az akasztó FA.,"Melyik az a növény amelyet, ha bent tartasz a ...","[Egészség, Egyéb kérdések]"
3,"[fibróma, eltávolítás, tépés]","Fibróma eltávolítás házilag, így szabad? Van a...","Ha tényleg az, nyugodtan leszedheted. Gyakran ...","Fibróma eltávolítás házilag, így szabad?","[Egészség, Betegségek]"
4,"[élet, víz, egészség, folyadék]",Ti meg tudtok inni egy nap minimum 2 lier vize...,"Igen, én megiszok annyit, de nem azért mert er...",Ti meg tudtok inni egy nap minimum 2 lier vizet?,"[Egészség, Táplálkozás]"


In [18]:
faq.rename(columns = {
        "valasz": "answer",
        "kategoriak": "categories",
        "hosszu_kerdes": "long_question",
        "rovid_kerdes": "short_question"
    }, inplace = True)


faq['main_category'] = faq.apply(lambda row: row.categories[0], axis = 1)
faq['sub_category'] = faq.apply(lambda row: row.categories[1], axis = 1)

faq.drop('categories', axis = 1, inplace = True)

faq.head()

,keywords,long_question,answer,short_question,main_category,sub_category
0,"[inzulinrezisztencia, diéta, reggeli]",Inzulinrezisztencia diétában mennyire kell kom...,Eléggé. Az ébredéstől egy órán belül meg kelle...,Inzulinrezisztencia diétában mennyire kell kom...,Egészség,Táplálkozás
1,"[nőgyógyászat, kivizsgálás, fájdalom, magánren...","Ha Borsod megyei vagyok, attól még mehetek mag...","Akárkihez mehetsz, akárhol.","Ha Borsod megyei vagyok, attól még mehetek mag...",Egészség,Nők egészsége
2,"[növény, halál]","Melyik az a növény amelyet, ha bent tartasz a ...",Az akasztó FA.,"Melyik az a növény amelyet, ha bent tartasz a ...",Egészség,Egyéb kérdések
3,"[fibróma, eltávolítás, tépés]","Fibróma eltávolítás házilag, így szabad? Van a...","Ha tényleg az, nyugodtan leszedheted. Gyakran ...","Fibróma eltávolítás házilag, így szabad?",Egészség,Betegségek
4,"[élet, víz, egészség, folyadék]",Ti meg tudtok inni egy nap minimum 2 lier vize...,"Igen, én megiszok annyit, de nem azért mert er...",Ti meg tudtok inni egy nap minimum 2 lier vizet?,Egészség,Táplálkozás


In [19]:
categories = set(faq.main_category)
print(categories)

{'Gyerekvállalás, nevelés', 'Szórakozás', 'Számítástechnika', 'Egészség', 'Állatok'}


In [20]:
for category in categories:
    category_size = faq[faq.main_category == category].shape[0]
    if category_size < args.min_questions_per_category:
        faq = faq[faq.main_category != category]

In [21]:
import unidecode
def unicode_to_ascii(data):
    return unidecode.unidecode(re.sub(r"[,.;@#?!&$]+\ *", " ", data).lower())

In [22]:
faq.long_question = faq.long_question.apply(lambda x: unicode_to_ascii(x))
faq.short_question = faq.short_question.apply(lambda x: unicode_to_ascii(x))
faq.answer = faq.answer.apply(lambda x: unicode_to_ascii(x))

/home/versenyi98/.local/lib/python3.6/site-packages/unidecode/__init__.py:50: RuntimeWarning: Surrogate character '\ud83d' will be ignored. You might be using a narrow Python build.
  return _unidecode(string)
/home/versenyi98/.local/lib/python3.6/site-packages/unidecode/__init__.py:50: RuntimeWarning: Surrogate character '\ude0f' will be ignored. You might be using a narrow Python build.
  return _unidecode(string)


In [23]:
by_main_category = collections.defaultdict(list)

for _, row in faq.iterrows():
    by_main_category[row.main_category].append(row.to_dict())

In [24]:
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_main_category.items()):
    np.random.shuffle(item_list)
    n = len(item_list)
    n_train = int(args.train_proportion * n)
    n_val = int(args.val_proportion * n)
    n_test = int(args.test_proportion * n)
    
    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:]:
        item['split'] = 'test'  
    
    final_list.extend(item_list)

In [25]:
final_faq = pd.DataFrame(final_list)

In [26]:
final_faq.split.value_counts()

train    117686
test      25223
val       25217
Name: split, dtype: int64

In [27]:
final_faq.head()

,keywords,long_question,answer,short_question,main_category,sub_category,split
0,"[here, herecsavarodás]",here torzio here csavarodas honnan tudhatom ho...,a herek meg osszekeveredni helyuket felcsereln...,here torzio here csavarodas honnan tudhatom ho...,Egészség,Férfiak egészsége,train
1,"[természetgyógyászat, kínai gyógyászat]",tenyleg olyan jo hatassal vannak az emberi ege...,elso vagyok eloszor is bocsanatot kerek az elo...,tenyleg olyan jo hatassal vannak az emberi ege...,Egészség,Bio életmód,train
2,"[mentális, betegség, randi, érzés, reakció]",szeretek jatszani az emberekkel vannak masok i...,az utolso egy paraszt szard le nyugi en is ezt...,szeretek jatszani az emberekkel vannak masok i...,Egészség,Mentális egészség,train
3,"[foghúzás, tömés, fúrás, orvos]",ha a korzeti fogorvos huzast mond akkor a szaj...,igy van menj el maganhoz az allapotfelmeres mi...,ha a korzeti fogorvos huzast mond akkor a\nsza...,Egészség,"Fogak, szájápolás",train
4,"[Béres csepp, zöld tea, tea]",a beres csepp reakcioba lephet a zoldteaval ma...,a tea csersavtartalmaval reakcioba lep a beres...,a beres csepp reakcioba lephet a zoldteaval,Egészség,Egyéb kérdések,train


In [28]:
final_faq.to_csv(args.output_munged_csv, index=False)